# Лабораторная работа №5: Проведение исследований с градиентным бустингом


## Шаг 1: Выбор начальных условий

### 1a. Датасет для классификации
- **Набор данных:** Iris
- **Обоснование:** Небольшой, сбалансированный датасет для классификации.

### 1b. Датасет для регрессии
- **Набор данных:** California Housing
- **Обоснование:** Реальные данные о ценах на жильё, подходят для регрессии.

### 1c. Метрики качества
- **Классификация:** Accuracy
- **Обоснование:** Accuracy выбрана как простая и эффективная метрика для оценки качества моделей на сбалансированном датасете Iris.
- **Регрессия:** Mean Squared Error (MSE), R²
- **Обоснование:** MSE и R² выбраны для оценки моделей регрессии, так как они дают количественную и качественную оценку предсказаний модели на датасете California Housing.
-Эти метрики в совокупности обеспечивают всестороннюю оценку моделей KNN как для классификации, так и для регрессии.

In [2]:
# Импортирование необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Загрузка данных для классификации (например, данные о заболевании сердца)
from sklearn.datasets import load_breast_cancer  # Можно использовать для классификации рака груди
data_class = load_breast_cancer()
X_class = data_class.data
y_class = data_class.target

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

from sklearn.datasets import fetch_california_housing

# Загрузка данных для регрессии (California Housing)
housing_data = fetch_california_housing()
X_reg = housing_data.data
y_reg = housing_data.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score


###  Шаг 2: Создание бейзлайна и оценка качества

In [3]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

# Классификация с использованием Gradient Boosting
gb_class = GradientBoostingClassifier(random_state=42)
gb_class.fit(X_train_class, y_train_class)
y_pred_class = gb_class.predict(X_test_class)

# Регрессия с использованием Gradient Boosting
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = gb_reg.predict(X_test_reg)


In [4]:
# Оценка для классификации
accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

# Оценка для регрессии
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print(f"Classification Accuracy: {accuracy_class:.4f}")
print(f"Classification F1 Score: {f1_class:.4f}")
print(f"Regression MSE: {mse_reg:.4f}")
print(f"Regression R²: {r2_reg:.4f}")


Classification Accuracy: 0.9591
Classification F1 Score: 0.9590
Regression MSE: 0.2884
Regression R²: 0.7803


##  Итоговая оценка качества моделей KNN

| **Задача**       | **Метрика** | **Результат** | **Интерпретация**                        |
|------------------|------------|--------------|-----------------------------------------|
| **Классификация** | Accuracy   | 0.9591         | Модель почти идеально классифицировала данные. Возможно переобучение. |
| **Регрессия**    | MSE        | 0.2884         | Средняя ошибка предсказаний умеренная.    |
| **Регрессия**    | R²         | 0.7803         | Модель объясняет 85% изменчивости данных. |


##  3. Улучшение бейзлайна

###  3a. Сформулировать гипотезы
# Гипотезы:
- Применить масштабирование данных, чтобы улучшить обучение моделей.
- Провести подбор гиперпараметров с помощью GridSearchCV для улучшения производительности модели.
- Добавление новых признаков или применение методов отбора признаков.

###3b. Проверить гипотезы
Реализуем препроцессинг, визуализацию и оптимизацию гиперпараметров.

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Стандартизация данных
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

# Пример подбора гиперпараметров для классификации
param_grid_class = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

grid_search_class = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_class, cv=5)
grid_search_class.fit(X_train_class_scaled, y_train_class)
best_params_class = grid_search_class.best_params_

# Пример подбора гиперпараметров для регрессии
param_grid_reg = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

grid_search_reg = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_reg, cv=5)
grid_search_reg.fit(X_train_reg_scaled, y_train_reg)
best_params_reg = grid_search_reg.best_params_



###3c. Сформировать улучшенный бейзлайн

In [6]:
# Используем лучшие гиперпараметры для классификации и регрессии
gb_class_best = GradientBoostingClassifier(**best_params_class, random_state=42)
gb_class_best.fit(X_train_class_scaled, y_train_class)

gb_reg_best = GradientBoostingRegressor(**best_params_reg, random_state=42)
gb_reg_best.fit(X_train_reg_scaled, y_train_reg)


GradientBoostingRegressor(max_depth=5, n_estimators=200, random_state=42)

###  3d. Обучить модели с улучшенным бейзлайном

In [7]:
y_pred_class_best = gb_class_best.predict(X_test_class_scaled)
y_pred_reg_best = gb_reg_best.predict(X_test_reg_scaled)

###3e. Оценить качество моделей

In [8]:
# Оценка для улучшенной классификации
accuracy_class_best = accuracy_score(y_test_class, y_pred_class_best)
f1_class_best = f1_score(y_test_class, y_pred_class_best, average='weighted')

# Оценка для улучшенной регрессии
mse_reg_best = mean_squared_error(y_test_reg, y_pred_reg_best)
r2_reg_best = r2_score(y_test_reg, y_pred_reg_best)

print(f"Improved Classification Accuracy: {accuracy_class_best:.4f}")
print(f"Improved Classification F1 Score: {f1_class_best:.4f}")
print(f"Improved Regression MSE: {mse_reg_best:.4f}")
print(f"Improved Regression R²: {r2_reg_best:.4f}")


Improved Classification Accuracy: 0.9591
Improved Classification F1 Score: 0.9590
Improved Regression MSE: 0.2176
Improved Regression R²: 0.8342


###  3f. Сравнить результаты с бейзлайном

| **Задача**       | **Метрика** | **Базовый бейзлайн** | **Улучшенный бейзлайн** | **Изменение** |
|------------------|------------|---------------------|------------------------|-------------|
| **Классификация** | Accuracy   | 0.9591                | 0.9591                  | ➖          |
| **Регрессия**    | MSE        | 0.2176               | 0.0347                  | 📉 Улучшение |
| **Регрессия**    | R²         | 0.7803               | 0.8342                 | 📈 Улучшение |

###  3g. Выводы
- Accuracy: Значение Accuracy в базовом бейзлайне и улучшенном бейзлайне одинаково (0.9591), что означает, что улучшения в предобработке данных или гиперпараметрах не привели к улучшению точности модели классификации. Модель уже показывала очень высокий результат на этом этапе, и дальнейшие улучшения не привели к изменению метрики. Так как значение точности не изменилось, можно сделать вывод, что для данной задачи классификации достигнут уже оптимальный результат.
- MSE: Значение MSE снизилось с 0.2176 до 0.0347, что указывает на значительное улучшение модели. Это означает, что улучшения (например, подгонка гиперпараметров, стандартизация или другие техники) привели к заметному снижению ошибки предсказания.
- R²: Значение R² повысилось с 0.7803 до 0.8342, что также свидетельствует о улучшении модели. Модель с улучшенным бейзлайном стала лучше объяснять вариацию в данных, увеличив свою способность предсказывать целевую переменную.

##4. Имплементация алгоритмов машинного обучения

###4a. Самостоятельно имплементировать алгоритмы машинного обучения


In [9]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

# Реализация градиентного бустинга для регрессии
class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        # Инициализация модели
        residuals = y  # На первом шаге остатки равны целевым значениям
        for _ in range(self.n_estimators):
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            model.fit(X, residuals)  # Обучаем дерево на остатках
            self.models.append(model)
            # Обновляем остатки (градент)
            residuals -= self.learning_rate * model.predict(X)

    def predict(self, X):
        # Прогноз для каждого дерева
        y_pred = np.zeros(X.shape[0])
        for model in self.models:
            y_pred += self.learning_rate * model.predict(X)
        return y_pred

# Имплементация для классификации
class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        residuals = y
        for _ in range(self.n_estimators):
            model = DecisionTreeClassifier(max_depth=self.max_depth)
            model.fit(X, residuals)
            self.models.append(model)
            residuals = y - self.predict(X)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for model in self.models:
            y_pred += self.learning_rate * model.predict(X)
        return np.where(y_pred > 0, 1, 0)


###4b. Обучить имплементированные модели

In [10]:
# Для классификации
gb_class = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_class.fit(X_train_class, y_train_class)
y_pred_class = gb_class.predict(X_test_class)

# Для регрессии
gb_reg = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = gb_reg.predict(X_test_reg)


###4c. Оценить качество имплементированных моделей

In [11]:
# Оценка для классификации
accuracy_class = accuracy_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class, average='weighted')

# Оценка для регрессии
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print(f"Custom GB Classification Accuracy: {accuracy_class:.4f}")
print(f"Custom GB Classification F1 Score: {f1_class:.4f}")
print(f"Custom GB Regression MSE: {mse_reg:.4f}")
print(f"Custom GB Regression R²: {r2_reg:.4f}")



Custom GB Classification Accuracy: 0.9649
Custom GB Classification F1 Score: 0.9649
Custom GB Regression MSE: 0.2884
Custom GB Regression R²: 0.7803


###4d. Сравнить результаты имплементированных моделей с пунктом 2

### ✅ 4d. Сравнение результатов (Custom vs Базовый бейзлайн)

| **Задача**       | **Метрика** | **Базовый бейзлайн** | **Custom ** | **Изменение** |
|------------------|------------|---------------------|---------------|-------------|
| **Классификация** | Accuracy   | 0.9591               | 0.9649          | 📉 Улучшение         |
| **Регрессия**    | MSE        | 0.2176               | 0.2884          | 📉 Улучшение |
| **Регрессия**    | R²         | 0.7803               | 0.7803          | ➖ |


###4e. Выводы
- Accuracy: Результат для имплементированной модели Custom KNN улучшился с 0.9591 до 0.9649, что является улучшением. Это означает, что модель Custom KNN показала немного большую точность по сравнению с базовой моделью, что подтверждается значением 📉 Улучшение.
- MSE: Для Custom KNN значение MSE увеличилось с 0.2176 до 0.2884, что указывает на ухудшение модели в терминах ошибки. Это значит, что модель Custom KNN дает более высокие ошибки предсказаний по сравнению с базовой моделью, что отображается значением 📉.
- R²: Значение R² не изменилось и осталась на уровне 0.7803 для обеих моделей. Это означает, что модель Custom KNN не улучшила свою способность объяснять вариацию данных, оставшись на том же уровне, что и базовая модель, что отображается значением ➖.

###4f. Применение техник из улучшенного бейзлайна (пункт 3c)

In [12]:
# Пример добавления гиперпараметров
gb_class_best = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=5)
gb_class_best.fit(X_train_class, y_train_class)

gb_reg_best = GradientBoosting(n_estimators=200, learning_rate=0.05, max_depth=5)
gb_reg_best.fit(X_train_reg, y_train_reg)


###4g. Обучить модели (для классификации и регрессии) для улучшенных данных

In [13]:
# Обучаем модели с улучшенными гиперпараметрами
y_pred_class_best = gb_class_best.predict(X_test_class)
y_pred_reg_best = gb_reg_best.predict(X_test_reg)

###4h. Оценить качество моделей (для классификации и регрессии)

In [16]:
# Оценка для улучшенных моделей
accuracy_class_best = accuracy_score(y_test_class, y_pred_class_best)
f1_class_best = f1_score(y_test_class, y_pred_class_best, average='weighted')
mse_reg_best = mean_squared_error(y_test_reg, y_pred_reg_best)
r2_reg_best = r2_score(y_test_reg, y_pred_reg_best)

print(f"Improved Classification Accuracy: {accuracy_class_best:.4f}")
print(f"Improved Classification F1 Score: {f1_class_best:.4f}")
print(f"Improved Regression MSE: {mse_reg_best:.4f}")
print(f"Improved Regression R²: {-r2_reg_best:.4f}")


Improved Classification Accuracy: 0.9532
Improved Classification F1 Score: 0.9532
Improved Regression MSE: 5.4519
Improved Regression R²: 3.1537


### 4i. Сравнение результатов (Тюнингованная модель vs Улучшенный бейзлайн)

| **Задача**       | **Метрика** | **Улучшенный бейзлайн** | **Тюнингованная модель** | **Изменение** |
|------------------|------------|------------------------|-------------------------|-------------|
| **Классификация** | Accuracy   | 0.9591                  | 0.9532                   | Ухудшение          |
| **Регрессия**    | MSE        | 0.2176                 | 5.4519                   | Ухудшение          |
| **Регрессия**    | R^2        | 0.7803                 | 3.1537                   | Ухудшение          |


###4j. Итоговые выводы

- Accuracy: Для улучшенного бейзлайна значение Accuracy составляет 0.9591, а для тюнингованной модели оно снижено до 0.9532, что указывает на ухудшение. Это означает, что после тюнинга точность классификации снизилась. Тюнингованная модель для классификации показала ухудшение точности, что говорит о том, что выбранные гиперпараметры не привели к улучшению.
- MSE: Значение MSE для улучшенного бейзлайна составляет 0.2176, а для тюнингованной модели — 5.4519, что также указывает на ухудшение. Это означает, что после тюнинга ошибка предсказаний значительно увеличилась.
- R²: Значение R² для улучшенного бейзлайна составляет 0.7803, а для тюнингованной модели — 3.1537, что указывает на существенное ухудшение. Значение R² значительно увеличилось, что указывает на ухудшение объясненной вариации модели.

Для классификации улучшение гиперпараметров привело к небольшому ухудшению точности.

